The goal of this python notebooks are :
- [x] listing all the json files
- [ ] computing the score of the list of aswers for all json files
- [ ] (*grouping json files that come from the same participants*)
- [ ] doing some test to detect :
  - [ ] missing values
  - [ ] experiment saved multiple times
  - [ ] experiment we forgot to save
- [ ] gathering everything into a pandas dataframe
- [ ] saving the dataframe for further analysis with potentially other softwares

# Imports

Don't forget to mount the drive.

In [ ]:
import pandas as pd
import numpy as np
import os
import json
import glob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Listing all the json files

In [ ]:
#To allow this line to run, add a shortut of the SCORE drive to your's
path = "/content/drive/MyDrive/SCORE - VR vision/Results/raw/json"

In [ ]:
jsonPathes = glob.glob(path + "/*.json")

# Opening the jsonfiles

In [ ]:
results = []
for jsonpath in jsonPathes :
  fileName = os.path.basename(jsonpath)

  with open(jsonpath) as jsonfile:
    result = json.load(jsonfile)

  if "VR" in fileName :
    result["VR"] = True
  else :
    result["VR"] = False

  results.append(result)

# Checking the values

To ease further preparation of the data, we could do some assumptions about the data we have. But we need to verify them before.

First of all, do all the datapoint contain one and only one pair for each slot ?

In [ ]:
for i,result in enumerate(results):
  counter = [0]*7
  for pair in result['Pairs']:
    counter[pair['slotIndex']] += 1
  if counter != [1]*7 :
    print(i)
    print(counter)
    print(result)


24
[1, 1, 1, 1, 1, 0, 2]
{'time': '4/29/2022 9:25:28 AM', 'experimentID': '11', 'chosenDataset': 1, 'Pairs': [{'slotIndex': 0, 'answer': 'true3(Clone)'}, {'slotIndex': 1, 'answer': 'true4(Clone)'}, {'slotIndex': 2, 'answer': 'true5(Clone)'}, {'slotIndex': 3, 'answer': 'false2(Clone)'}, {'slotIndex': 4, 'answer': 'true7(Clone)'}, {'slotIndex': 6, 'answer': 'true2(Clone)'}, {'slotIndex': 6, 'answer': 'true6(Clone)'}], 'InspectionTimes': [{'observationTime': 7.616520404815674, 'name': 'true1(Clone)'}, {'observationTime': 2.616600513458252, 'name': 'true2(Clone)'}, {'observationTime': 8.499849319458008, 'name': 'true3(Clone)'}, {'observationTime': 2.4166438579559326, 'name': 'true4(Clone)'}, {'observationTime': 3.133286714553833, 'name': 'true5(Clone)'}, {'observationTime': 8.299856185913086, 'name': 'true6(Clone)'}, {'observationTime': 2.0666329860687256, 'name': 'true7(Clone)'}, {'observationTime': 2.4166419506073, 'name': 'false1(Clone)'}, {'observationTime': 10.5831298828125, 'name': '

We can see that 2 experiment have this problem. As the two experiment were not on VR, this can be explained by the fact that bug in our software caused the "validate button" to appear early. For the experiment of participant 11, it seems that the participants stacked two answers on the same slot, which is still possible because of some bug in our software.

Because participant 11 put two shapes in one slot, we don't know which one they really wanted, os we will remove this value.

However, for the rest of the analysis, we cannot assume all slot were full.

In [ ]:
results[24]['Pairs'] = results[24]['Pairs'][:5]

-------------------

Another thing to test : do all the experiment have an experiment ID and do all participant ID corresond to preciselly 4 experiment.

In [ ]:
for result in results:
  if not result["experimentID"].isnumeric():
    print(result)

all the result have a correct ID

-------------------------

let's test if there are 4 results per participant ID. There should be 25 participants with ID ranging from 1 to 25.

In [ ]:
counter = [0]*25

for result in results:
  counter[ int(result['experimentID'])-1 ] += 1

[(Id+1, count) for Id,count in enumerate(counter) if count != 4]

[(2, 3), (4, 1), (13, 3), (16, 2), (23, 0)]

We can see that 5 values are problematic :
 - participant 2 : a bug in our software prevented us to save the last results
 - participant 4 : same bug as participant 2 + we were not able to do the the VR part
 - participant 13 : no idea. check with coco's notes
 - participant 16 : first experiment, the protocol was different, we need to delete this line
 - participant 23 : the participant didn't participate


 ----------------------------


 We also need to check that all and only experiment not performed on VR all have values for the observation time

In [ ]:
for result in results:
  if (result["VR"] and len(result["InspectionTimes"])!=0) or ((not result["VR"]) and len(result["InspectionTimes"])!=14):
    print(result)
    print(len(result["InspectionTimes"]))
    print(result["VR"])

{'time': '4/27/2022 4:14:36 PM', 'experimentID': '10', 'chosenDataset': 1, 'Pairs': [{'slotIndex': 0, 'answer': 'false3(Clone)'}, {'slotIndex': 1, 'answer': 'false4(Clone)'}, {'slotIndex': 2, 'answer': 'true4(Clone)'}, {'slotIndex': 3, 'answer': 'false2(Clone)'}, {'slotIndex': 4, 'answer': 'true7(Clone)'}, {'slotIndex': 5, 'answer': 'true6(Clone)'}, {'slotIndex': 6, 'answer': 'false5(Clone)'}], 'InspectionTimes': [], 'pauseTime': 0.0, 'sortingTime': 0.0, 'VR': False}
0
False


these values were entered by hand

In [ ]:
def TrueProp(dataFrame) -> float:
  # return the proportion of true answers
	return sum(dataFrame['isTrue'])/len(dataFrame)

def VeracityScoring(answ) -> int:
	total_dist = index = 0
	answLen = len(answ)
	worstNote = answLen*answLen
	for i in range(len(answ)):
		if IsTrue(answ[i]):
			index = Index(answ[i])-1
			total_dist += abs(i - index)
		else:
			total_dist += answLen
  #Return this score as a percentage
	return 100 * (worstNote - total_dist)/worstNote

def ConsecutiveFalses(answ):
	index = [0] * len(answ)
	score = 0
	for i in range(len(answ)):
		index[i] = 0 if IsTrue(answ[i]) else Index(answ[i])
		score += 1 if (index[i-1]+1==index[i]) else 0
	return score

classicDistScore = {
    'sub' : 1,
    'del' : 1,
    'ins' : 1
}

def levenDistIter(seq1,seq2,distScore):
  a = np.zeros((len(seq1)+1,len(seq2)+1), dtype = np.int32)
  for i in range(len(seq1)+1): # for each element of the matrix
    for j in range(len(seq2)+1): 
      if i == 0 : # first line
        a[i][j]=j
      elif j == 0 : # first column
        a[i][j]=i
      elif seq1[i-1]==seq2[j-1]: # identity
        a[i][j]=a[i-1][j-1]
      else: # mismatch
        a[i][j]=np.min([a[i-1][j]+ distScore['del'], #deletion
                        a[i][j-1] + distScore['ins'] , # insertion
                        a[i-1][j-1] + distScore['sub']]) # substitution
  #print(a)
  return a[-1][-1]

# Applying the metrics to each row

In [ ]:
results2 = results.copy()

for result in results2:
  for obsTime in result["InspectionTimes"]:
    name = obsTime["name"].split("(")[0]
    result["ObsTime " + name] = obsTime["observationTime"]

  result["answ"] = [""] * 7
  for pair in result["Pairs"] :
    result["answ"][pair["slotIndex"]] = pair["answer"].split("(")[0]
    result["Slot" + str(pair["slotIndex"])] = pair["answer"].split("(")[0]
  
  result["trueProp"] = TrueProp(result["answ"])

  result["veracityScoring"] = VeracityScoring(result["answ"])
  #result["consecutiveFalses"] = ConsecutiveFalses(result["answ"])

  result["levensteinDist"] = levenDistIter(result["answ"],["true" + str (i+1) for i in range(7)], classicDistScore)

# deleting useless rows

In [ ]:
for result in results2:

  result.pop("InspectionTimes")
  result.pop("Pairs")

In [ ]:
resultDf = pd.DataFrame(results2)

In [ ]:
resultDf

,time,experimentID,chosenDataset,pauseTime,sortingTime,VR,answ,Slot0,Slot1,Slot2,...,ObsTime true5,ObsTime true6,ObsTime true7,ObsTime false1,ObsTime false2,ObsTime false3,ObsTime false4,ObsTime false5,ObsTime false6,ObsTime false7
0,4/27/2022 1:36:32 PM,16,1,184.909470,269.096130,True,"[true4, true2, false2, true6, true5, true7, tr...",true4,true2,false2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4/27/2022 1:49:19 PM,16,2,342.400177,221.314514,True,"[true4, true6, false3, false7, false6, true1, ...",true4,true6,false3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4/27/2022 2:43:34 PM,13,2,34.628632,304.130951,True,"[false4, true4, false2, false7, true5, true6, ...",false4,true4,false2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4/27/2022 2:54:16 PM,13,1,162.141891,227.080627,True,"[false3, true1, true2, true7, false2, true6, t...",false3,true1,true2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4/27/2022 3:13:53 PM,13,1,370.547791,247.132828,False,"[false3, false2, false1, false7, true1, true2,...",false3,false2,false1,...,6.783242,26.382708,4.216589,16.533073,7.416524,14.349827,8.066503,13.416476,4.716595,4.366597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,5/2/2022 5:26:31 PM,24,2,5.297991,173.299805,True,"[false1, true4, true5, false3, true1, false6, ...",false1,true4,true5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85,5/2/2022 6:10:06 PM,15,1,61.760620,104.243004,True,"[false2, false6, false1, true5, false4, false7...",false2,false6,false1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,5/2/2022 6:15:28 PM,15,2,2.555197,114.941154,True,"[false7, true2, false1, false3, true4, true3, ...",false7,true2,false1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87,5/2/2022 6:29:12 PM,15,2,9.416518,210.855438,False,"[false6, true4, true6, true3, false5, false7, ...",false6,true4,true6,...,6.249920,12.699805,5.983225,7.449885,4.183298,5.949911,6.249885,2.883271,11.216486,3.566616


In [ ]:
resultDf.to_csv("clean_result.csv")

# Participant to delete

There were methodological problem with some participants, we will need to delete their results.